In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.autograd.forward_ad as fwAD

primal = torch.randn(10, 10)
tangent = torch.randn(10, 10)

def fn(x, y):
    return x ** 2 + y ** 2

# All forward AD computation must be performed in the context of
# a ``dual_level`` context. All dual tensors created in such a context
# will have their tangents destroyed upon exit. This is to ensure that
# if the output or intermediate results of this computation are reused
# in a future forward AD computation, their tangents (which are associated
# with this computation) won't be confused with tangents from the later
# computation.
with fwAD.dual_level():
    # To create a dual tensor we associate a tensor, which we call the
    # primal with another tensor of the same size, which we call the tangent.
    # If the layout of the tangent is different from that of the primal,
    # The values of the tangent are copied into a new tensor with the same
    # metadata as the primal. Otherwise, the tangent itself is used as-is.
    #
    # It is also important to note that the dual tensor created by
    # ``make_dual`` is a view of the primal.
    dual_input = fwAD.make_dual(primal, tangent)
    assert fwAD.unpack_dual(dual_input).tangent is tangent

    # To demonstrate the case where the copy of the tangent happens,
    # we pass in a tangent with a layout different from that of the primal
    dual_input_alt = fwAD.make_dual(primal, tangent.T)
    assert fwAD.unpack_dual(dual_input_alt).tangent is not tangent

    # Tensors that do not have an associated tangent are automatically
    # considered to have a zero-filled tangent of the same shape.
    plain_tensor = torch.randn(10, 10)
    dual_output = fn(dual_input, plain_tensor)

    # Unpacking the dual returns a ``namedtuple`` with ``primal`` and ``tangent``
    # as attributes
    jvp = fwAD.unpack_dual(dual_output).tangent

assert fwAD.unpack_dual(dual_output).tangent is None

In [10]:
from tensordict import TensorDict

In [58]:
primal = TensorDict({'a': torch.arange(30).reshape(10, 3).float(), 'b': torch.ones(10, 4, 5)}, batch_size=10)

In [59]:
tangent = TensorDict({'a': torch.ones(10, 3), 'b': torch.ones(10, 4, 5)}, batch_size=10)

In [60]:
def f_dict(x):
    return x['a'] ** 2 + x['b'].sum()

In [61]:
torch.func.jvp(f_dict, (primal,), (tangent,))

(tensor([[ 200.,  201.,  204.],
         [ 209.,  216.,  225.],
         [ 236.,  249.,  264.],
         [ 281.,  300.,  321.],
         [ 344.,  369.,  396.],
         [ 425.,  456.,  489.],
         [ 524.,  561.,  600.],
         [ 641.,  684.,  729.],
         [ 776.,  825.,  876.],
         [ 929.,  984., 1041.]]),
 tensor([[200., 202., 204.],
         [206., 208., 210.],
         [212., 214., 216.],
         [218., 220., 222.],
         [224., 226., 228.],
         [230., 232., 234.],
         [236., 238., 240.],
         [242., 244., 246.],
         [248., 250., 252.],
         [254., 256., 258.]]))

In [62]:
from tensordict import TensorDict
import torch

# 2つのtensordictを作成
tensordict1 = TensorDict({
    "key1": torch.randn(3, 4),
    "key2": torch.randn(3, 4)
}, batch_size=[3])

tensordict2 = TensorDict({
    "key1": torch.randn(3, 4),
    "key2": torch.randn(3, 4)
}, batch_size=[3])

# 2つのtensordictを足す
result = tensordict1 + tensordict2

# 結果を確認
print(result["key1"])
print(result["key2"])

TypeError: unsupported operand type(s) for +: 'TensorDict' and 'TensorDict'

In [64]:
1.0 * tensordict1

TypeError: unsupported operand type(s) for *: 'float' and 'TensorDict'

In [65]:
tensordict1.shape

torch.Size([3])

In [66]:
type(tensordict1)

tensordict._td.TensorDict

In [67]:
tensordict1.apply(lambda x: 2 * x)

TensorDict(
    fields={
        key1: Tensor(shape=torch.Size([3, 4]), device=cpu, dtype=torch.float32, is_shared=False),
        key2: Tensor(shape=torch.Size([3, 4]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([3]),
    device=None,
    is_shared=False)

In [72]:
tensordict1.apply(lambda x: 2 * x)["key2"]

tensor([[ 1.0484, -1.8561,  1.4272,  1.0533],
        [-1.5890,  0.2027, -0.1566,  1.5146],
        [ 1.2385,  2.0889,  1.0955, -1.2020]])

In [71]:
tensordict1['key2']

tensor([[ 0.5242, -0.9280,  0.7136,  0.5267],
        [-0.7945,  0.1013, -0.0783,  0.7573],
        [ 0.6193,  1.0444,  0.5478, -0.6010]])

In [76]:
def add_tensordicts(tensordict1, tensordict2):
    """
    2つのtensordictを足し合わせる関数
    
    Args:
        tensordict1 (TensorDict): 1つ目のtensordict
        tensordict2 (TensorDict): 2つ目のtensordict
        
    Returns:
        TensorDict: 2つのtensordictを足し合わせた結果
    """
    # 2つのtensordictのキーが同じであることを確認
    assert set(tensordict1.keys()) == set(tensordict2.keys()), "Keys in the two tensordicts must be the same."
    
    # 各keyに対応するtensorを足し合わせる
    result = {k:tensordict1[k] + tensordict2[k] for k in tensordict1.keys()}
    
    # 新しいtensordictを作成して返す
    return TensorDict(result, batch_size=tensordict1.batch_size)

In [84]:
add_tensordicts(tensordict1, tensordict2)["key2"]

tensor([[ 0.3702,  0.1352,  0.4325,  1.0031],
        [-1.2659, -1.4851,  0.7063, -0.8183],
        [ 0.7257,  0.3506,  0.5705, -0.2927]])

In [83]:
tensordict1["key2"] + tensordict2["key2"]

tensor([[ 0.3702,  0.1352,  0.4325,  1.0031],
        [-1.2659, -1.4851,  0.7063, -0.8183],
        [ 0.7257,  0.3506,  0.5705, -0.2927]])

In [86]:
isinstance(tensordict1, torch.Tensor)

False

In [1]:
from nigbms.utils.solver import rademacher_like

: 

In [2]:
import torch
from petsc4py import PETSc
from tensordict import TensorDict

[1713701388.815420] [bae72f51ae96:24547:0]     ucp_context.c:1774 UCX  WARN  UCP version is incompatible, required: 1.16, actual: 1.12 (release 1)
[1713701388.829896] [bae72f51ae96:24547:0]     ucp_context.c:1774 UCX  WARN  UCP version is incompatible, required: 1.16, actual: 1.12 (release 1)


/home/sohei/.local/lib/python3.10/site-packages/tensordict/_pytree.py:147: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  register_pytree_node(


In [3]:
import torch

In [4]:
from tensordict import TensorDict

In [59]:
tensordict = TensorDict({'a': torch.arange(30).reshape(10, 3).float(), 'b': torch.ones(10, 2, 5)}, batch_size=10)

In [60]:
def f(x):
    return x['a'].sum() + x['b'].sum()

In [61]:
tensordict * b.reshape(10, 1)

RuntimeError: scalar tensor expected to be 0 dim but it has 2 dimensions and 10 elements.

In [62]:
y = f(tensordict)

In [63]:
a = torch.ones(10) * 2
b = torch.randn(10)

In [64]:
(tensordict * 0.1)["a"]

tensor([[0.0000, 0.1000, 0.2000],
        [0.3000, 0.4000, 0.5000],
        [0.6000, 0.7000, 0.8000],
        [0.9000, 1.0000, 1.1000],
        [1.2000, 1.3000, 1.4000],
        [1.5000, 1.6000, 1.7000],
        [1.8000, 1.9000, 2.0000],
        [2.1000, 2.2000, 2.3000],
        [2.4000, 2.5000, 2.6000],
        [2.7000, 2.8000, 2.9000]])

In [65]:
tensordict["a"]

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.],
        [15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]])

In [ ]:
def 

In [66]:
tensordict.apply(lambda x: x * a)["b"]

RuntimeError: The size of tensor a (3) must match the size of tensor b (10) at non-singleton dimension 1

In [67]:
import torch

def multiply_tensor(tensor1, tensor2):
    """
    (N, 1)のtensorを(N, a_1, a_2, ..., a_n)のtensorに乗算する関数
    
    Args:
        tensor1 (torch.Tensor): (N,)のtensor
        tensor2 (torch.Tensor): (N, a_1, a_2, ..., a_n)のtensor
    
    Returns:
        torch.Tensor: (N, a_1, a_2, ..., a_n)のtensor
    """
    # tensor1を適切な形状に変形する
    for _ in range(tensor2.ndim - 1):
        tensor1 = tensor1.unsqueeze(-1)
    
    # 乗算を実行
    result = tensor1 * tensor2
    
    return result

In [68]:
# 入力データの例
tensor1 = torch.randn(10, 1)
tensor2 = torch.randn(10, 5, 3)

# 関数を呼び出して結果を取得
result = multiply_tensor(tensor1, tensor2)
print(result.shape)  # Output: torch.Size([10, 5, 3])

torch.Size([10, 10, 5, 3])


In [75]:
# 入力Tensor
a = torch.randn(10)
b = torch.randn(10, 3, 4, 5)

# Aをreshape
a_reshaped = reshape_a_to_b(a, b)
print(a_reshaped.size())  # torch.Size([10, 1, 1, 1])

torch.Size([10, 1, 1, 1])


In [99]:
s[1:] = 1

In [100]:
s

tensor([10,  1,  1,  1])